In [1]:
import json
import random
import os
from pathlib import Path
from typing import List, Dict, Any
import shutil
import time

# ==============================================================================
# CONFIGURAÇÕES
# ==============================================================================

# Diretório onde estão os JSONs criados pelo extrator_cenipa_llm.py
# (Ajuste se o seu caminho de saída for diferente)
INPUT_DIR = Path('data/json/llm_ready')

# Novo diretório de destino para os relatórios selecionados (holdout set)
HOLDOUT_DIR = Path('data/json/holdout_set')

# Critérios de seleção
MIN_FATORES_CONTRIBUINTES = 3  # Mínimo de fatores contribuintes que o JSON deve ter
NUM_AMOSTRA_DESEJADA = 100     # Quantidade de documentos a selecionar

# ==============================================================================
# FUNÇÕES DE SELEÇÃO E MOVIMENTAÇÃO
# ==============================================================================

def carregar_e_filtrar_arquivos(input_dir: Path) -> List[Dict[str, Any]]:
    """
    Carrega todos os arquivos JSON, filtra os que atendem ao critério de fatores,
    e retorna uma lista de dicionários contendo o caminho e o conteúdo.
    """
    arquivos_candidatos = []
    arquivos_json = list(input_dir.glob("*.json"))

    print(f"📁 Total de arquivos JSON encontrados em {input_dir}: {len(arquivos_json)}")

    for arquivo_path in arquivos_json:
        try:
            with open(arquivo_path, 'r', encoding='utf-8') as f:
                data = json.load(f)

            # Ajuste a chave de acesso conforme a estrutura do arquivo gerado
            # Usando a estrutura do extrator_cenipa_llm.py: data['conteudo']['fatores_contribuintes']
            fatores_contribuintes = data.get('conteudo', {}).get('fatores_contribuintes', [])

            num_fatores = len(fatores_contribuintes)

            if num_fatores >= MIN_FATORES_CONTRIBUINTES:
                # Armazena o caminho e o número de fatores
                arquivos_candidatos.append({
                    'path': arquivo_path,
                    'num_fatores': num_fatores
                })

        except Exception as e:
            print(f"  ❌ Erro ao ler ou analisar {arquivo_path.name}: {e}")

    print(f"🎯 Total de relatórios com >= {MIN_FATORES_CONTRIBUINTES} fatores: {len(arquivos_candidatos)}")
    return arquivos_candidatos

def selecionar_e_mover_holdout(arquivos_candidatos: List[Dict[str, Any]], num_amostra: int):
    """
    Seleciona a amostra aleatória, cria o diretório de destino e move os arquivos.
    """
    if not arquivos_candidatos:
        print("🛑 Nenhuma amostra candidata encontrada. Finalizando.")
        return

    # 1. Seleciona a amostra
    amostra_final = []

    if len(arquivos_candidatos) <= num_amostra:
        print(f"⚠️ Aviso: O número de candidatos ({len(arquivos_candidatos)}) é menor ou igual ao desejado ({num_amostra}). Selecionando todos.")
        amostra_final = arquivos_candidatos
    else:
        print(f"🎲 Selecionando amostra aleatória de {num_amostra} documentos...")
        amostra_final = random.sample(arquivos_candidatos, num_amostra)

    # 2. Cria o diretório de destino
    HOLDOUT_DIR.mkdir(parents=True, exist_ok=True)

    # 3. Move os arquivos
    movidos = 0
    erros = 0

    for item in amostra_final:
        src_path = item['path']
        dest_path = HOLDOUT_DIR / src_path.name

        try:
            # Move o arquivo para a nova pasta
            shutil.move(str(src_path), str(dest_path))
            movidos += 1
        except Exception as e:
            print(f"  ❌ Erro ao mover {src_path.name}: {e}")
            erros += 1

    print("\n" + "="*40)
    print("✅ PROCESSO DE HOLDOUT CONCLUÍDO")
    print(f"   Amostra selecionada: {movidos} arquivos")
    print(f"   Movidos para: {HOLDOUT_DIR.resolve()}")
    print(f"   Erros de movimentação: {erros}")
    print(f"   Total de JSONs restantes em {INPUT_DIR}: {len(list(INPUT_DIR.glob('*.json')))}")
    print("="*40)

# ==============================================================================
# EXECUÇÃO PRINCIPAL
# ==============================================================================

def main_holdout():
    print("🚀 INICIANDO SELEÇÃO DO HOLDOUT SET")
    print(f"   Critério: Mínimo de {MIN_FATORES_CONTRIBUINTES} fatores contribuintes.")
    print(f"   Amostra desejada: {NUM_AMOSTRA_DESEJADA} documentos.")

    # Carrega e filtra
    candidatos = carregar_e_filtrar_arquivos(INPUT_DIR)

    # Seleciona e move
    selecionar_e_mover_holdout(candidatos, NUM_AMOSTRA_DESEJADA)

if __name__ == "__main__":
    main_holdout()

🚀 INICIANDO SELEÇÃO DO HOLDOUT SET
   Critério: Mínimo de 3 fatores contribuintes.
   Amostra desejada: 100 documentos.
📁 Total de arquivos JSON encontrados em data\json\llm_ready: 1567
🎯 Total de relatórios com >= 3 fatores: 490
🎲 Selecionando amostra aleatória de 100 documentos...

✅ PROCESSO DE HOLDOUT CONCLUÍDO
   Amostra selecionada: 100 arquivos
   Movidos para: C:\Users\matos\OneDrive\Área de Trabalho\Programacao\Cenipa-ReportAnalysis\data\json\holdout_set
   Erros de movimentação: 0
   Total de JSONs restantes em data\json\llm_ready: 1467
